In [ ]:
import datetime
import rasterio as rs

def get_date(fl):
    
    year = fl.split('__')[-1].split('.')[0][11:15]
    month = fl.split('__')[-1].split('.')[0][15:17]
    day = fl.split('__')[-1].split('.')[0][17:19]
    
    return '%s-%s-%s'%(year,month,day)

def get_WaterYear(dt):
    
    strt = datetime.datetime(dt.year,10,1)
    nd = datetime.datetime(dt.year,12,31)
    
    if (dt >= strt) & (dt <= nd):
        return dt.year+1
    else:
        return dt.year
    
def DOWY(date):
    '''compute day of water year from a YYYY-MM-DD date string'''
    if date == 'None':
        return np.NaN
    else:
        year = int(date.split('-')[0])
        month = int(date.split('-')[1])
        day = int(date.split('-')[2])
        
        date = datetime.datetime(year,month,day)

        if date <= datetime.datetime(year,9,30):
            return (date - datetime.datetime(year - 1,10,1)).days
        else: 
            return (date - datetime.datetime(year,10,1)).days + 1
        
def get_dons(vals,dates=[]):
    '''Pull the date of no snow'''
    if np.nansum(vals) > 0:
        dons = dates[np.nanargmin(vals)] # return the first date that the pack hits 0
        return DOWY(str(dons.date()))
    else: return np.NaN
    
def get_peakSWE(vals):
    '''Pull the peak SWE value'''
    
    if np.nansum(vals) > 0:
        peakSWE = np.nanmax(vals)
        return peakSWE

    else:
        return np.NaN

def get_doPeakSWE(vals,dates=[]):
    '''Pull the date of peak SWE
    return a datetime object
    '''
    if np.nansum(vals) > 0:
        dops = dates[np.nanargmax(vals)] # return the first date that the pack maxes out
        return DOWY(str(dops.date()))
    
    else: return np.NaN

In [ ]:
# generate a list of file names
files = pd.DataFrame()
files['name'] = glob.glob('./data/SNODAS_SWE/SnowWaterEquivalent/*/*.tif')
files['datetime'] = files.name.map(get_date)
files.index = pd.DatetimeIndex(files.datetime)
del files['datetime']
files['WaterYear'] = files.index.map(get_WaterYear)
files.sort_index(ascending=True,inplace=True)

In [ ]:
waterYears = files.WaterYear.unique()

In [ ]:
#wy = 2006
wy = 2004
fileList = files.loc[files.WaterYear==wy] # extract the files necessary
n = len(fileList) # number of files or raster stack depth
dates = fileList.index

# load a test file to get the array dimensions
with rs.open(fileList.name[0]) as ds:
    j,k = ds.read(1).shape
    profile = ds.profile
    
tmp = np.ndarray((n,j,k)) # preallocate the array
tmp[:,:,:] = np.NaN # fill with NaNs
    
i = 0
for fl in fileList.name:

    with rs.open(fl) as ds:
        d = ds.read(1)

        jj,kk = d.shape
        if (jj != j) | (kk != k):
            print('Arrays are not the same shape, skipping slice %s'%i)
            del d
            i += 1
            continue
        else:
            dd = np.array(d,dtype = np.float32)
            del d
            dd[dd==-9999] = np.NaN
            tmp[i,:,:] = dd/1000. # pull the data into the stack and scale it
            i += 1
    
# preallocate
dowy_PeakSWE = np.ndarray((j,k),dtype=np.float32)
dowy_PeakSWE[:,:] = np.NaN
dowy_DONS = dowy_PeakSWE.copy()
PeakSWE = dowy_PeakSWE.copy()

for i in range(j):
    for o in range(k):
        vals = tmp[:,i,o]
        
        dowy_PeakSWE[i,o] = get_doPeakSWE(vals,dates=dates)
        dowy_DONS[i,o] = get_dons(vals,dates=dates)
        PeakSWE[i,o] = get_peakSWE(vals)

In [ ]:
plt.imshow(dowy_PeakSWE,cmap='Blues')
plt.colorbar()

In [ ]:
plt.imshow(dowy_DONS,cmap='Blues')
plt.colorbar()

In [ ]:
plt.imshow(PeakSWE,cmap='Blues')
plt.colorbar()

In [ ]:
print(np.nanmean(dowy_PeakSWE))
print(np.nanmean(dowy_DONS))
print(np.nanmean(PeakSWE))